In [1]:
from pynq import Overlay
from pynq import MMIO
import numpy as np
import time

ol=Overlay("layerAll.bit")
mmio=MMIO(0x43C00000,0x10000)

coloursBaseAddress= 0x040
orderBaseAddress = 0x200
moveArrayBaseAddress = 0x100

In [6]:
moves= []

def move_val_to_str (move_val):
    if(move_val==0):
        moves.append("U")
    elif(move_val==1):
        moves.append("U'")
    elif(move_val==2):
        moves.append("D")
    elif(move_val==3):
        moves.append("D'")
    elif(move_val==4):
        moves.append("B")
    elif(move_val==5):
        moves.append("B'")
    elif(move_val==6):
        moves.append("F")
    elif(move_val==7):
        moves.append("F'")
    elif(move_val==8):
        moves.append("L")
    elif(move_val==9):
        moves.append("L'")
    elif(move_val==10):
        moves.append("R")
    elif(move_val==11):
        moves.append("R'")
    elif(move_val==12):
        moves.append("H")
    elif(move_val==13):
        moves.append("V")
    elif(move_val==14):
        moves.append("H'")
    elif(move_val==15):
        moves.append("V'")
    else:
        print("error in converting move value to string")

def move_hex_to_str (move_hex):
    move_val_to_str(move_hex & 0xff) 
    move_val_to_str(move_hex >> 8 & 0xff)
    move_val_to_str(move_hex >> 16 & 0xff)
    move_val_to_str(move_hex >> 24 & 0xff)
    
def col_ord(colour):
    if(colour == "W"):
        return 0
    elif (colour=="Y"):
        return 1
    elif (colour=="O"):
        return 2
    elif (colour=="R"):
        return 3
    elif (colour=="G"):
        return 4
    elif (colour=="B"):
        return 5
    else:
        print ("error in converting colour to num")

def solve (colours):
    for i in np.arange(0,52,4):
        coloursMem = col_ord(colours[i]) + (col_ord(colours[i+1])<< 8) + (col_ord(colours[i+2])<< 16) +(col_ord(colours[i+3])<< 24)
        mmio.write(coloursBaseAddress+ i ,coloursMem)
    coloursMem= col_ord(colours[52]) +(col_ord(colours[53])<< 8)
    mmio.write(coloursBaseAddress+ 52 ,coloursMem)
    mmio.write(0x000,1)
    start_time = time.time()
    while ((mmio.read(0x000)& 0x2)!=2):
        pass
    elapsed_time = time.time() - start_time
    
    order= mmio.read(orderBaseAddress)
    i =0
    while( i<order):   
        move_hex = mmio.read(moveArrayBaseAddress+i)
        move_hex_to_str (move_hex)
        i=i+4
    print("Total number of moves are %d" %order)
    print("elapsed time is ", elapsed_time)
    print(moves[0:order])


In [7]:
solve("RBBGWOWWOYRWYYWYWBRYGYOGYROOGGWROBBRYOBRGBGRRWYWBBGGOO")
solve("WWWWWWWWWYYYYYYYYYOOOOOOOOORRRRRRRRRGGGGGGGGGBBBBBBBBB")
solve("GGGOWOGGGBBBRYRBBBYYYOOOYYYWWWRRRWWWRGRYGYRGROBOWBWOBO")
solve("WRGRWORBGBGGYYBRGWYYBWOGRRYBWWRRWYOWOBYYGWGBOOYOOBGROB")

Total number of moves are 165
elapsed time is  0.0002663135528564453
["F'", 'D', 'R', 'R', 'F', "R'", 'U', "F'", "U'", 'H', 'R', 'R', 'H', 'B', 'B', "D'", 'R', 'R', 'H', 'D', 'D', 'R', 'R', "R'", "D'", 'R', 'H', 'D', 'D', "R'", "D'", 'R', 'D', "R'", "D'", 'R', 'D', "R'", "D'", 'R', 'D', 'H', 'H', 'H', "R'", "D'", 'R', 'H', 'H', "D'", "R'", "D'", 'R', 'D', "R'", "D'", 'R', 'D', "R'", "D'", 'R', 'D', 'H', "R'", "D'", 'R', 'D', 'H', "D'", "R'", "D'", 'R', 'D', 'H', 'V', 'V', 'H', "U'", "L'", 'U', 'L', 'U', 'F', "U'", "F'", 'H', "U'", "U'", "L'", 'U', 'L', 'U', 'F', "U'", "F'", 'H', 'U', "L'", 'U', 'L', 'U', 'F', "U'", "F'", 'H', "L'", 'U', 'L', 'U', 'F', "U'", "F'", 'H', 'F', 'R', 'U', "R'", "U'", "F'", 'R', 'U', "R'", 'U', 'R', 'U', 'U', "R'", "R'", 'F', "R'", 'B', 'B', 'R', "F'", "R'", 'B', 'B', 'R', 'R', 'H', "R'", 'F', "R'", 'B', 'B', 'R', "F'", "R'", 'B', 'B', 'R', 'R', "U'", 'F', 'F', "U'", 'L', "R'", 'F', 'F', "L'", 'R', "U'", 'F', 'F']
Total number of moves are 3
elapsed time is  

In [21]:
cubieColor= [
[ "W","W","W","W","W","W","W","W","W" ], 
[ "Y","Y","Y","Y","Y","Y","Y","Y","Y" ], 
[ "O","O","O","O","O","O","O","O","O" ], 
[ "R","R","R","R","R","R","R","R","R" ], 
[ "G","G","G","G","G","G","G","G","G" ], 
[ "B","B","B","B","B","B","B","B","B" ]  
]

print("Total number of moves are %d" %order)
print(moves[0:order])

#still do not remove the move by "H" and "V" which is just cube's horizontal and vertical rotation
#will be remove in later stage 
#actual move after delete "H" and "V" is 0

Total number of moves are 3
['V', 'V', 'H']


In [42]:
cubieColor= [
[ "R","B","B","G","W","O","W","W","O" ], 
[ "Y","R","W","Y","Y","W","Y","W","B" ], 
[ "R","Y","G","Y","O","G","Y","R","O" ], 
[ "O","G","G","W","R","O","B","B","R" ], 
[ "Y","O","B","R","G","B","G","R","R" ], 
[ "W","Y","W","B","B","G","G","O","O" ]  
]

print("Total number of moves are %d" %order)
print(moves[0:order])
#actual move after delete "H" and "V" is 145

Total number of moves are 165
["F'", 'D', 'R', 'R', 'F', "R'", 'U', "F'", "U'", 'H', 'R', 'R', 'H', 'B', 'B', "D'", 'R', 'R', 'H', 'D', 'D', 'R', 'R', "R'", "D'", 'R', 'H', 'D', 'D', "R'", "D'", 'R', 'D', "R'", "D'", 'R', 'D', "R'", "D'", 'R', 'D', 'H', 'H', 'H', "R'", "D'", 'R', 'H', 'H', "D'", "R'", "D'", 'R', 'D', "R'", "D'", 'R', 'D', "R'", "D'", 'R', 'D', 'H', "R'", "D'", 'R', 'D', 'H', "D'", "R'", "D'", 'R', 'D', 'H', 'V', 'V', 'H', "U'", "L'", 'U', 'L', 'U', 'F', "U'", "F'", 'H', "U'", "U'", "L'", 'U', 'L', 'U', 'F', "U'", "F'", 'H', 'U', "L'", 'U', 'L', 'U', 'F', "U'", "F'", 'H', "L'", 'U', 'L', 'U', 'F', "U'", "F'", 'H', 'F', 'R', 'U', "R'", "U'", "F'", 'R', 'U', "R'", 'U', 'R', 'U', 'U', "R'", "R'", 'F', "R'", 'B', 'B', 'R', "F'", "R'", 'B', 'B', 'R', 'R', 'H', "R'", 'F', "R'", 'B', 'B', 'R', "F'", "R'", 'B', 'B', 'R', 'R', "U'", 'F', 'F', "U'", 'L', "R'", 'F', 'F', "L'", 'R', "U'", 'F', 'F']


In [25]:
cubieColor= [
[ "G","G","G","O","W","O","G","G","G" ], 
[ "B","B","B","R","Y","R","B","B","B" ], 
[ "Y","Y","Y","O","O","O","Y","Y","Y" ], 
[ "W","W","W","R","R","R","W","W","W" ], 
[ "R","G","R","Y","G","Y","R","G","R" ], 
[ "O","B","O","W","B","W","O","B","O" ]  
]

print("Total number of moves are %d" %order)
print(moves[0:order])
#actual move after delete "H" and "V" is 130

Total number of moves are 157
['D', 'R', 'U', "F'", "U'", 'H', 'D', 'R', 'U', "F'", "U'", 'H', 'B', 'B', "D'", 'R', 'U', "F'", "U'", 'H', "R'", "R'", 'U', "F'", "U'", 'H', 'D', 'D', "R'", "D'", 'R', 'D', 'H', 'H', "D'", "R'", "D'", 'R', 'D', 'H', "R'", "D'", 'R', 'D', 'H', 'V', 'V', 'H', "L'", 'U', 'L', 'U', 'F', "U'", "F'", 'H', 'U', 'R', "U'", "R'", "U'", "F'", 'U', 'F', 'H', 'H', 'H', 'R', "U'", "R'", "U'", "F'", 'U', 'F', 'H', 'U', 'R', "U'", "R'", "U'", "F'", 'U', 'F', 'H', 'H', 'F', 'U', 'R', "U'", "R'", "F'", 'H', 'H', 'H', 'R', 'U', "R'", 'U', 'R', 'U', 'U', 'U', "R'", 'U', 'R', 'U', 'U', "R'", 'H', 'H', 'R', 'U', "R'", 'U', 'R', 'U', 'U', "R'", 'U', 'U', 'H', 'H', 'H', "R'", 'F', "R'", 'B', 'B', 'R', "F'", "R'", 'B', 'B', 'R', 'R', "U'", 'H', 'F', 'F', "U'", 'L', "R'", 'F', 'F', "L'", 'R', "U'", "U'", 'L', "R'", 'F', 'F', "L'", 'R', "U'", 'F', 'F']


In [29]:
cubieColor= [
[ "W","R","G","R","W","O","R","B","G" ], 
[ "B","G","G","Y","Y","B","R","G","W" ], 
[ "Y","Y","B","W","O","G","R","R","Y" ], 
[ "B","W","W","R","R","W","Y","O","W" ], 
[ "O","B","Y","Y","G","W","G","B","O" ], 
[ "O","Y","O","O","B","G","R","O","B" ]  
]

print("Total number of moves are %d" %order)
print(moves[0:order])
#actual move after delete "H" and "V" is 152

Total number of moves are 192
['F', 'F', 'D', 'R', 'U', "F'", "U'", 'H', 'D', 'D', 'R', 'U', "F'", "U'", 'H', 'R', 'R', 'H', "F'", 'D', 'R', 'R', 'F', 'H', 'H', 'H', "R'", "D'", 'R', 'H', 'H', "D'", "R'", "D'", 'R', 'D', 'H', 'H', 'H', "R'", "D'", 'R', 'H', 'H', "D'", "R'", "D'", 'R', 'D', "R'", "D'", 'R', 'D', "R'", "D'", 'R', 'D', 'H', 'H', "R'", "D'", 'R', 'H', 'H', 'H', "R'", "D'", 'R', 'D', "R'", "D'", 'R', 'D', "R'", "D'", 'R', 'D', "R'", "D'", 'R', 'D', "R'", "D'", 'R', 'D', 'H', 'H', 'V', 'V', 'H', 'H', 'H', 'U', 'R', "U'", "R'", "U'", "F'", 'U', 'F', 'H', 'H', 'H', "U'", "U'", "L'", 'U', 'L', 'U', 'F', "U'", "F'", 'H', 'H', 'U', 'R', "U'", "R'", "U'", "F'", 'U', 'F', 'H', 'H', 'U', 'R', "U'", "R'", "U'", "F'", 'U', 'F', 'H', 'H', "L'", 'U', 'L', 'U', 'F', "U'", "F'", 'U', 'U', 'R', "U'", "R'", "U'", "F'", 'U', 'F', 'H', 'H', 'F', 'U', 'R', "U'", "R'", "F'", 'R', 'U', "R'", 'U', 'R', 'U', 'U', "R'", 'H', 'R', 'U', "R'", 'U', 'R', 'U', 'U', "R'", 'U', 'U', 'H', 'H', 'H', 'F', 'F